notes : 
- trouver comment exploiter le "superuser"

In [12]:
import pandas as pd
import json

In [9]:
data = []
with open("data/philly_restaurant_reviews.json", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
textes=pd.DataFrame(data)

In [11]:
textes.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
1,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
2,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
3,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0,0,0,0,Tremendous service (Big shout out to Douglas) ...,2013-06-24 11:21:25
4,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,0,0,0,My absolute favorite cafe in the city. Their b...,2014-11-12 15:30:27


Nous n’avons volontairement pas concaténé l’ensemble des avis d’un restaurant afin d’éviter un biais de volume lié au nombre d’avis. 

In [ ]:
import pandas as pd

# nombre d'avis par business
nb_avis = textes.groupby("business_id").size()

nb_avis.describe()


count    5852.000000
mean      117.445147
std       247.806219
min         5.000000
25%        14.000000
50%        40.000000
75%       118.000000
max      5778.000000
dtype: float64

Nous prenons 40 avis par restaurant ce qui correspond à la mediane du nombre d'avis par restaurant. Nous prenons les 40 avis les plus recents pour avoir l'information la plus a jour.

In [15]:
# s'assurer que la colonne date est bien au format datetime
textes["date"] = pd.to_datetime(textes["date"])

textes_restaurant = (
    textes
    .sort_values("date", ascending=False)   # du plus récent au plus ancien
    .groupby("business_id")
    .head(40)                                # garder les 40 plus récents
    .groupby("business_id", as_index=False)["text"]
    .apply(" ".join)
)

textes_restaurant.head()

,business_id,text
0,-0M0b-XhtFagyLmsBtOe8w,Ok so this place is an extremely odd hole in t...
1,-0PN_KFPtbnLQZEeb23XiA,Far and away the best Chinese food to order if...
2,-0TffRSXXIlBYVbb5AwfTg,I went to indieblue a few years ago for happy ...
3,-0eUa8TsXFFy0FCxHYmrjg,"We are sitting in an ICU waiting room, several..."
4,-1B9pP_CrRBJYPICE5WbRA,The food is absolutely amazing. I get food ...


Conformément à la méthodologie proposée par BERTopic, les documents sont représentés à l’aide d’embeddings Sentence-BERT (SBERT), une approche spécifiquement conçue pour comparer des phrases et des documents entiers sur la base de leur similarité sémantique. d'apres les auteurs it achieves state-of-the-art performance on various sentence embedding tasks (Reimers and urevych, 2020; Thakur et al., 2020). En particulier, le modèle all-MiniLM-L6-v2 est recommandé dans des contextes à ressources limitées sans GPU

notes : donner + d'argument prk sbert bon pour notre cas

Sbert doit recevoir un texte le plus priche du langage naturel donc les preprocessing lourds comme les stopword removal ou la lemmatization sont appliqués suelement dans la partie topic representations (c-TF-IDF) suivant les recos (Grootendorst (2022) and Reimers & Gurevych (2019).)

In [16]:
import re
import html

def clean_for_embedding(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # Decode HTML entities
    text = html.unescape(text)

    # Remove URLs
    text = re.sub(r"http\S+|www\S+", " ", text)

    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)

    # Trim
    text = text.strip()

    # Optional but fine
    text = text.lower()

    return text
textes_restaurant["cleaned_text_embedding"] = textes_restaurant["text"].apply(clean_for_embedding)

On peut passer maintenant àl'embedding 

In [18]:
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer("all-MiniLM-L6-v2")
texts = textes_restaurant["cleaned_text_embedding"].tolist()
embeddings = model.encode(
    texts,
    batch_size=32,          
    show_progress_bar=True,
    normalize_embeddings=True  #ce sera pratique pour le clustering plus tard
)

/Users/taniaadmane/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/taniaadmane/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 183/183 [00:33<00:00,  5.42it/s]


In [19]:
embeddings

array([[ 0.04560389, -0.00026394,  0.01085451, ..., -0.01332529,
        -0.11193341, -0.02507696],
       [ 0.00807339,  0.02861315,  0.10805283, ...,  0.03106915,
        -0.11294399,  0.04018402],
       [ 0.04750501, -0.05986113,  0.06890885, ..., -0.01268979,
        -0.0664725 , -0.01671202],
       ...,
       [-0.04140325,  0.02234362,  0.08899155, ..., -0.03159462,
        -0.10733926,  0.00531767],
       [-0.01924057, -0.00993413,  0.03526418, ..., -0.01455262,
        -0.0129594 , -0.04179877],
       [-0.01764687,  0.00970904,  0.11258573, ..., -0.02093395,
        -0.03934057,  0.01052198]], dtype=float32)